# 設定ファイルの編集--GPU(GRES)の登録

---

OpenHPC環境の設定ファイルの内容を変更します。

## 概要

このNotebookでは、GPUなどをSlurmの[Generic Resource(GRES)](https://slurm.schedmd.com/gres.html)として追加する手順を示します。

GRESは Graphics Processing Units (GPUs), CUDA Multi-Process Service (MPS), and Intel® Many Integrated Core (MIC) processors を Slurm で扱うためのプラグインです。デフォルト構成のSlurmでは
GRESが利用できないので、このNobookで設定ファイルの変更を行いGRESを利用できるようにします。

編集対象とするのは以下の２つの設定ファイルです。

* /etc/slurm/slurm.conf
* /etc/slurm/gres.conf

![処理の流れ](images/ohpc-010.png)

このNotebookの主な手順は以下のようになります。

1. マスターノードの設定ファイルをNotebook環境に取得する
2. 取得したファイルのバックアップを作成する
3. Notebookの編集機能を利用して設定ファイルの変更をおこなう
4. 変更した設定ファイルを各VCノードに配置する
5. 各VCノードのSLURMデーモンに設定ファイルの再読み込みを指示する

## 前提条件

このNotebookを実行するための前提条件を満たしていることを確認します。

以下のことを前提条件とします。
* 構築済のOpenHPC環境がある
* 各VCノードに対してAnsibleで操作できるように設定されている
* 各VCノードに対して管理者権限で操作が行える

VCノードを作成時に指定した値を確認するために `group_vars` ファイル名の一覧を表示します。

In [ ]:
!ls -1 group_vars/*.yml | sed -e 's/^group_vars\///' -e 's/\.yml//' | sort

操作対象となる UnitGroup 名を指定してください。

In [ ]:
# (例)
# ugroup_name = 'OpenHPC'

ugroup_name = 

疎通確認を行います。

In [ ]:
!ansible {ugroup_name} -m ping 

管理者権限でコマンドが実行できることを確認します。

In [ ]:
!ansible {ugroup_name} -b -a 'whoami'

## 設定ファイルの編集

OpenHPC環境の設定ファイルを取得して、Jupyter Notebookの編集機能を用いて設定ファイルを編集します。

### slurm.conf

Slurmの設定ファイル`slurm.conf`を編集します。

> 設定ファイルの記述方法の詳細については [slurm.conf(5)](https://slurm.schedmd.com/slurm.conf.html)を参照してください。

次のセルを実行すると、以下の手順が実行されます。

1. マスターノードの設定ファイル(`slurm.conf`)をローカル環境に取得する
2. 取得した設定ファイルのバックアップを作成する
3. Jupyter Notebookの編集機能を利用して設定ファイルを編集するためのリンクを表示する

ローカル環境に取得したファイルは、以下のパスに格納されています。

`./edit/{ugroup_name}/{YYYYMMDDHHmmssffffff}/slurm.conf`

`{ugroup_name}` には UnitGroup名が、`{YYYYMMDDHHmmssfffff}` にはファイルを取得したタイムスタンプが入ります。

また、バックアップファイルは以下のパスに格納されます。

`./edit/{ugroup_name}/{YYYYMMDDHHmmssffffff}/slurm.conf.orig`

In [ ]:
%run scripts/edit_conf.py
slurm_conf_path = '/etc/slurm/slurm.conf'
fetch_conf(slurm_conf_path, ugroup_name, 'master')

上のセルに表示されているリンクをクリックするとファイルの編集画面が開きます。必要な箇所の編集を行い、その結果をファイルに保存してください。ファイルの保存を行うにはメニューの File -- Save を選択するか、キーボードで Ctrl-S をタイプしてください。

GPUをGRESとして登録するには [`GresTypes`の設定](https://slurm.schedmd.com/slurm.conf.html#OPT_GresTypes)と、[ノード設定](https://slurm.schedmd.com/slurm.conf.html#SECTION_NODE-CONFIGURATION)に `Gres` を追加する必要があります。例えばc1-c4に一つのGPUをGRESとして設定するには以下のような記述を追加します。

```
NodeName=c[1-4] Gres=gpu:1 CPUs=4 Boards=1 SocketsPerBoard=1 CoresPerSocket=2 ThreadsPerCore=2 RealMemory=15751 State=UNKNOWN
GresTypes=gpu
```

次のセルを実行すると、計算ノードの実際のリソース量を確認することができます。`slurm.conf`を編集する際の参考にしてください。

In [ ]:
!ansible {ugroup_name}_compute -a '/usr/sbin/slurmd -C'

`slurm.conf`の`Gres`を記述する際の参考のために、計算ノードのGPUのリストを表示します。

In [ ]:
!ansible {ugroup_name}_compute -a 'nvidia-smi -L'

設定ファイルの編集後に次のセルを実行すると、編集の前後での差分を確認することができます。

In [ ]:
%run scripts/edit_conf.py
show_local_conf_diff(slurm_conf_path, ugroup_name, 'master')

### gres.conf

`/dev/nvidia0`などのデバイスファイルとの関連付けを行うために `gres.conf` を編集します。

次のセルを実行すると`gres.conf`を編集するリンクが表示されます。

In [ ]:
%run scripts/edit_conf.py
gres_conf_path = '/etc/slurm/gres.conf'
!ansible {ugroup_name}_master -m file -b \
    -a 'path={gres_conf_path} state=touch'
fetch_conf(gres_conf_path, ugroup_name, 'master')

`/dev/nvidia0` を関連付けるには以下のような記述を追加します。

```
Name=gpu File=/dev/nvidia0
```

`gres.conf`の設定内容の詳細については [gres.conf(5)](https://slurm.schedmd.com/gres.conf.html) を参照してください。

設定ファイルの編集後に次のセルを実行すると、編集の前後での差分を確認することができます。

In [ ]:
%run scripts/edit_conf.py
show_local_conf_diff(gres_conf_path, ugroup_name, 'master')

## 編集した設定ファイルの反映

編集したファイルをVCノードに配置して、設定ファイルの変更内容をコンテナに反映させます。

まず、変更前のノードの状態を確認しておきます。

In [ ]:
!ansible {ugroup_name}_master -a 'scontrol show node'

では、実際に設定ファイルの変更を行います。

次のセルを実行すると、`slurm.conf`, `gres.conf` のそれぞれの設定ファイルについて以下の手順が実行されます。

1. 編集前と編集後の設定ファイルの差分を表示する
2. 編集した設定ファイルを各VCノードに配置する

In [ ]:
%run scripts/edit_conf.py
upload_conf(slurm_conf_path, ugroup_name)
upload_conf(gres_conf_path, ugroup_name)

変更後の設定を反映させるためにマスターノード、計算ノードの SLURMデーモンを再起動します。

In [ ]:
!ansible {ugroup_name}_master -b -a 'systemctl restart slurmctld'

In [ ]:
!ansible {ugroup_name}_compute -b -a 'systemctl restart slurmd'

ノードの状態を確認します。

In [ ]:
!ansible {ugroup_name}_master -a sinfo

`STATE` が `drain` となっている場合はノード状態の更新が必要となります。次のセルのコメント(先頭の `#`)を外して実行してください。

In [ ]:
# %run scripts/group.py
# gvars = load_group_vars(ugroup_name)
# for x in gvars['compute_etc_hosts'].values():
#     !ansible {ugroup_name}_master -b -a 'scontrol update nodename={x} state=resume' || true

更新後の状態を再度確認します。

In [ ]:
!ansible {ugroup_name}_master -a sinfo

各ノードのGRESの状況を表示してみます。

In [ ]:
!ansible {ugroup_name}_master -a 'sinfo --Node --Format=NodeHost,Gres,GresUsed'

ただしく設定変更が行われていると`GRES`の欄にGPUが表示されます。`c1`から`c4`の各ノードに１つのGPUがある場合の表示例を以下に示します。`GRES`の列に GPU が表示されていることを確認してください。
```
HOSTNAMES           GRES             GRES_USED           
c1                  gpu:1            gpu:0
c2                  gpu:1            gpu:0
c3                  gpu:1            gpu:0
c4                  gpu:1            gpu:0
```

ノード状態の詳細を表示してみます。

In [ ]:
!ansible {ugroup_name}_master -a 'scontrol show node'

表示されたノードの状態が正しくない場合は「4. 設定ファイルの編集」以降のセルを unfreeze してから再度設定ファイルの変更を行ってください。